In [1]:
a =  0.4420450177378683
b = 0.010690629668597906
c = 0.060119161083589945

x = a + b + c + 1.0
w1 = a / x
w2 = (a + b) / x
w3 = (a + b + c) / x
w4 = 1.0
weights = [w1, w2, w3, w4]
print(weights)

[0.29219262500085036, 0.29925915221060156, 0.338998035774447, 1.0]


In [12]:
# assume you already have step_2_losses from your find_min_s2_loss logic
# e.g.:
import re

log_path = '/home/cherish/SADD/sfd-main/exps/00054-cifar10-4-3-dpmpp-3-poly7.0/log.txt'
all_losses = [[] for _ in range(3)]
pattern = re.compile(r"Step:\s*(\d+)\s*\|\s*Loss-mean:\s*([0-9]+\.[0-9]+)")
with open(log_path, 'r') as f:
    for line in f:
        m = pattern.search(line)
        if m:
            step = int(m.group(1))
            loss = float(m.group(2))
            if 0 <= step < 3:
                all_losses[step].append(loss)

step_2_losses = all_losses[2]

# now find the min and its iteration index
min_loss = min(step_2_losses)
min_iter = step_2_losses.index(min_loss)
print(f"Min Step-2 loss {min_loss:.6f} occurred at iteration index {min_iter}")

Min Step-2 loss 4.244495 occurred at iteration index 1133


In [10]:
find_min_s2_loss("/home/cherish/SADD/sfd-main/exps/00054-cifar10-4-3-dpmpp-3-poly7.0/log.txt")

1563


4.24449539

In [4]:
def find_min_s2_loss(log_path: str) -> float:
    """
    Parse the log file to extract all Step 2 Loss_ls-3-mean values
    and return their minimum.
    """
    
    step2_losses = []
    
    # Pattern to match Step 2 Loss_ls-3-mean lines
    step2_loss_pattern = re.compile(r"Step:\s*2\s*\|\s*Loss-mean:\s*([0-9]+\.[0-9]+)")

    
    with open(log_path, 'r') as f:
        for line in f:
            match = step2_loss_pattern.search(line)
            if match:
                loss_value = float(match.group(1))
                step2_losses.append(loss_value)
                # print(f"Found Step 2 Loss_ls-3-mean: {loss_value}")
    
    if not step2_losses:
        raise ValueError("No Step 2 Loss_ls-3-mean values found")
    
    min_loss = min(step2_losses)
    # print(f"\nAll Step 2 Loss_ls-3-mean values: {step2_ls3_losses}")
    # print(f"Total count: {len(step2_ls3_losses)}")
    # print(f"Minimum Step 2 Loss_ls-3-mean: {min_loss}")
    
    return min_loss

In [5]:
import re
find_min_s2_loss("/teamspace/studios/this_studio/sfd-main/training/original_training_logs.txt")

4.69043016

In [8]:
import pickle

pkl_path = '/teamspace/studios/this_studio/sfd-main/exps/teacher_model/edm-cifar10-32x32-cond-vp.pkl'  # replace with your .pkl file path
with open(pkl_path, 'rb') as f:
    data = pickle.load(f)

if hasattr(data, 'keys'):
    print("Keys in the pickle file:", list(data.keys()))
else:
    print(f"Loaded object is a {type(data)}, which has no .keys()")

Keys in the pickle file: ['ema', 'loss_fn', 'augment_pipe', 'dataset_kwargs']


In [9]:
# Print the state_dict of the model inside the loaded pickle
state_dict = data['ema'].model.state_dict()
print("Model state_dict keys and tensor shapes:")
for name, tensor in state_dict.items():
    print(f"{name}: {tuple(tensor.shape)}")

Model state_dict keys and tensor shapes:
map_label.weight: (128, 10)
map_label.bias: (128,)
map_augment.weight: (128, 9)
map_layer0.weight: (512, 128)
map_layer0.bias: (512,)
map_layer1.weight: (512, 512)
map_layer1.bias: (512,)
enc.32x32_conv.weight: (128, 3, 3, 3)
enc.32x32_conv.bias: (128,)
enc.32x32_block0.norm0.weight: (128,)
enc.32x32_block0.norm0.bias: (128,)
enc.32x32_block0.conv0.weight: (256, 128, 3, 3)
enc.32x32_block0.conv0.bias: (256,)
enc.32x32_block0.affine.weight: (256, 512)
enc.32x32_block0.affine.bias: (256,)
enc.32x32_block0.norm1.weight: (256,)
enc.32x32_block0.norm1.bias: (256,)
enc.32x32_block0.conv1.weight: (256, 256, 3, 3)
enc.32x32_block0.conv1.bias: (256,)
enc.32x32_block0.skip.weight: (256, 128, 1, 1)
enc.32x32_block0.skip.bias: (256,)
enc.32x32_block1.norm0.weight: (256,)
enc.32x32_block1.norm0.bias: (256,)
enc.32x32_block1.conv0.weight: (256, 256, 3, 3)
enc.32x32_block1.conv0.bias: (256,)
enc.32x32_block1.affine.weight: (256, 512)
enc.32x32_block1.affine.bi